In [1]:
using PorousMaterials
## file paths for crystal files and fragment files
@eval PorousMaterials PATH_TO_CRYSTALS = pwd() 
# fragment_location = joinpath(pwd(),"fragments")

"/home/ng/DTRA"

In [2]:
# # add MyModule to the path that Julia uses for modules 
push!(LOAD_PATH, joinpath(pwd()))

4-element Array{String,1}:
 "@"                      
 "@v#.#"                  
 "@stdlib"                
 "/home/ng/MOFfun.jl/src/"

In [3]:
using MOFun

┌ Info: Precompiling MOFun [top-level]
└ @ Base loading.jl:1273
  ** incremental compilation may be fatally broken for this module **



In [4]:
# The name of the fragment
# file must be a .xyz file located in ./fragments/
# The atom species that is bonded to the :C_aro_R atom on the ring segment of the Fragment
fragment_list = ["Br", "CH2-CH2-CH3", "CH2-CH3", "CH3", 
                "Cl", "C-N", "F", "NH2", "N-NH", 
                "O-CH2-CH2-CH3", "O-CH2-CH3", "O-CH3", "OH"]

# The type of Arene Substitution ("ortho", "meta", "all") 
# TODO: "all" not yet implimented
substitution_types = ["ortho", "meta"]

2-element Array{String,1}:
 "ortho"
 "meta" 

In [5]:
# function choose_side(randomize_side::Bool=true; set_side::Int=2)
#     # The side of the ring you wish to functionalize
#     # must be an Int (if substitution_position == "all" this parameter won't matter)
#     if randomize_side
#         return rand((1, 2))
#     elseif !randomize_side
#         return set_side
#     end
# end

choose_side (generic function with 2 methods)

In [5]:
# which_side = choose_side(random_side, set_side=1)

ErrorException: function choose_side does not accept keyword arguments

In [7]:
# select the crystal you wish to sunctionalize
# which_crystal = "SBMOF-1.cif"
which_crystal = "NiPyC2.cif"
# which_crystal = "IRMOF-1_clean.cssr"
@info which_crystal


# open crystal in lower level symmetry
crystal = Crystal(which_crystal, convert_to_p1=false)
# crystal = replicate(Crystal(which_crystal, convert_to_p1=true), (2, 2, 2))

strip_numbers_from_atom_labels!(crystal)
write_xyz(crystal)
bonding_rules = [BondingRule(:H, :*, 0.4, 1.2),
                 BondingRule(:N, :Ni, 0.4, 2.5),
                 BondingRule(:O, :Ni, 0.4, 2.5),
                 BondingRule(:Ca, :O, 0.4, 2.5),
                 BondingRule(:Zn, :O, 0.4, 2.5),
                 BondingRule(:Br, :*, 0.4, 2.5),
                 BondingRule(:*, :*, 0.4, 1.9)]

infer_bonds!(crystal, true, bonding_rules) 
write_bond_information(crystal, remove_extension(crystal) * "_bonds.vtk")
@debug bond_sanity_check(crystal) 

# The species of the functional atom (R) and the ipso atom of the aromatic ring
r_species = :blah
ipso_species = :blah
if which_crystal == "NiPyC2.cif"
    r_species = :Ni
    ipso_species = :N
elseif which_crystal == "SBMOF-1.cif"
    r_species = :S
    ipso_species = :C
elseif which_crystal == "IRMOF-1_clean.cssr"
    r_species = :C
    ipso_species = :C
else
    error("don't know this crystal")
end

┌ Info: NiPyC2.cif
└ @ Main In[7]:5


Saving bond information for crystal NiPyC2.cif to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2_bonds.vtk.


:N

In [9]:
for fragment_name in fragment_list
    for sub_type in substitution_types
        if sub_type != "all"
            functionalize_mof(crystal, fragment_name, ipso_species, r_species,
                                bonding_rules; n=6, randomize_side=true,
                                arene_substitution_type=sub_type)
        elseif sub_type =="all"
            # do something special
            continue
        else
            error("substituation type not supported")
        end
        
    end
endcd 

Saving bond information for crystal NiPyC2_NiPyC2_aligned_Br_NiPyC2_aligned_Br to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2/NiPyC2_ortho_functonalized_Br_bonds.vtk.
Saving bond information for crystal NiPyC2_NiPyC2_aligned_Br_NiPyC2_aligned_Br to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2/NiPyC2_meta_functonalized_Br_bonds.vtk.
Saving bond information for crystal NiPyC2_NiPyC2_aligned_CH2-CH2-CH3_NiPyC2_aligned_CH2-CH2-CH3 to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2/NiPyC2_ortho_functonalized_CH2-CH2-CH3_bonds.vtk.
Saving bond information for crystal NiPyC2_NiPyC2_aligned_CH2-CH2-CH3_NiPyC2_aligned_CH2-CH2-CH3 to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2/NiPyC2_meta_functonalized_CH2-CH2-CH3_bonds.vtk.
Saving bond information for crystal NiPyC2_NiPyC2_aligned_CH2-CH3_NiPyC2_aligned_CH2-CH3 to /home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/NiPyC2/NiPyC2_ortho_functonalized_CH2-CH3_bonds.vtk.
Saving bond inform

In [10]:
# functionalize_mof(xtal::Crystal, fragment_name::String, ipso_species::Symbol, r_species::Symbol,
#                                 bonding_rules::Array{BondingRule,1}; n::Int=6,
#                                 side_to_functionalize::Int=2, randomize_side::Bool=true,
#                                 arene_substitution_type::String="para")